<a href="https://colab.research.google.com/github/AlexChin-dev/La_Redoute/blob/main/La_Redoute_sql_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## База для работы


In [ ]:
import sqlite3
import pandas as pd
conn_sqlite = sqlite3.connect('TestDB.db')
c = conn_sqlite.cursor()
#Создание локальной БД
request = """
CREATE TABLE Client_info
    ([ClientID] varchar(6), [City] varchar(50))
;
INSERT INTO Client_info
    ([ClientID] , [City])
VALUES
    ( '822353', 'Kazan'),
    ( '694330', 'Kazan'),
    ( '412830', 'Kazan'),
    ( '722464', 'Kazan'),
    ( '303510', 'Kazan'),
( '327730', 'Sankt-Peterburg'),
( '974994', 'Sankt-Peterburg'),
( '501716', 'Sankt-Peterburg'),
( '400730', 'Sankt-Peterburg'),
( '760556', 'Sankt-Peterburg'),
( '866797', 'Sankt-Peterburg'),
( '934230', 'Sankt-Peterburg'),
( '494558', 'Sankt-Peterburg'),
( '517881', 'Sankt-Peterburg'),
( '863773', 'Moscow'),
( '802312', 'Moscow'),
( '746379', 'Moscow'),
( '271567', 'Moscow'),
( '955631', 'Moscow'),
( '466287', 'Moscow'),
( '539442', 'Moscow'),
( '372476', 'Moscow'),
( '433381', 'Moscow'),
( '602353', 'Moscow'),
( '815976', 'Moscow'),
( '713649', 'Moscow'),
( '932794', 'Moscow'),
( '877365', 'Moscow')
;
CREATE TABLE Order_info
    ([OrderID] varchar(9), [ClientID] varchar(6),  [Item] varchar(30))
;
INSERT INTO Order_info
    ([OrderID] , [ClientID], [Item])
VALUES
( '850840535', '822353', 'Dress'),
( '850840535', '822353', 'Socks'),
( '850840535', '822353', 'Curtains'),
( '708501413', '694330', 'Jacket'),
( '708501413', '694330', 'Pants'),
( '348586676', '412830', 'Sofa'),
( '288654482', '327730', 'Dress'),
( '288654482', '327730', 'Pants'),
( '288654482', '327730', 'Jacket'),
( '288654482', '327730', 'Socks'),
( '750884288', '974994', 'Dress'),
( '750884288', '974994', 'Pants'),
( '750884288', '974994', 'Sofa'),
( '750884288', '974994', 'Curtains'),
( '463930571', '501716', 'Curtains'),
( '463930571', '501716', 'Sofa'),
( '652237630', '400730', 'Jacket'),
( '652237630', '400730', 'Socks'),
( '652237630', '400730', 'Curtains'),
( '628409403', '955631', 'Bed'),
( '628409403', '955631', 'Curtains'),
( '509824566', '466287', 'Table'),
( '509824566', '466287', 'Bed'),
( '867337795', '539442', 'Pants'),
( '867337795', '539442', 'Jacket'),
( '867337795', '539442', 'Socks'),
( '527756777', '372476', 'Dress'),
( '527756777', '372476', 'Pants'),
( '527756777', '372476', 'Jacket'),
( '841157684', '433381', 'Curtains'),
( '841157684', '433381', 'Sofa'),
( '841157684', '433381', 'Pants'),
( '344921087', '602353', 'Dress'),
( '344921087', '602353', 'Jacket'),
( '344921087', '602353', 'Socks')
;
CREATE TABLE Items_Fashion
    ([Item] varchar(30), [Cost] numeric(8,2))
;
INSERT INTO Items_Fashion
    ([Item] , [Cost])
VALUES
( 'Dress', '4999'),
( 'Pants', '3500'),
( 'Jacket', '9000'),
( 'Socks', '200')
;
CREATE TABLE Items_Furniture
    ([Item] varchar(30), [Cost] numeric(8,2))
;
INSERT INTO Items_Furniture
    ([Item] , [Cost])
VALUES
( 'Sofa', '42999'),
( 'Curtains', '8000'),
( 'Table', '15500'),
( 'Bed', '25300')
;

"""
c.executescript(request)

## Разбор по частям 


In [ ]:
#cчитаем кол-во клиентов в каждом городе 
df = pd.read_sql('''
select count(distinct clientid), city from client_info
group by city
''', conn_sqlite)
df

,count(distinct clientid),City
0,5,Kazan
1,14,Moscow
2,9,Sankt-Peterburg


In [ ]:
#считаем товары заказанные по условиям задачи 
#>5000 одежда и мебель >20000 cтоимость
df = pd.read_sql('''
select * from client_info c
join order_info o on o.clientid = c.clientid 
join items_fashion i on i.item = o.item 
where i.cost > 5000
union all
select * from client_info c
join order_info o on o.clientid = c.clientid 
join items_furniture it on it.item = o.item 
where it.cost > 20000
''', conn_sqlite)
df

,ClientID,City,OrderID,ClientID,Item,Item,Cost
0,327730,Sankt-Peterburg,288654482,327730,Jacket,Jacket,9000
1,602353,Moscow,344921087,602353,Jacket,Jacket,9000
2,372476,Moscow,527756777,372476,Jacket,Jacket,9000
3,400730,Sankt-Peterburg,652237630,400730,Jacket,Jacket,9000
4,694330,Kazan,708501413,694330,Jacket,Jacket,9000
5,539442,Moscow,867337795,539442,Jacket,Jacket,9000
6,412830,Kazan,348586676,412830,Sofa,Sofa,42999
7,501716,Sankt-Peterburg,463930571,501716,Sofa,Sofa,42999
8,974994,Sankt-Peterburg,750884288,974994,Sofa,Sofa,42999
9,433381,Moscow,841157684,433381,Sofa,Sofa,42999


In [ ]:
#cчитаем кол-во заказов в каждом городе для каждого условия
df = pd.read_sql('''
select count(o.orderid) as count, c.city from client_info c
join order_info o on o.clientid = c.clientid 
join items_fashion i on i.item = o.item 
where i.cost > 5000
group by c.city
union all
select count(o.orderid) as count, c.city from client_info c
join order_info o on o.clientid = c.clientid 
join items_furniture it on it.item = o.item 
where it.cost > 20000
group by c.city
''', conn_sqlite)
df

,count,City
0,1,Kazan
1,3,Moscow
2,2,Sankt-Peterburg
3,1,Kazan
4,3,Moscow
5,2,Sankt-Peterburg


In [ ]:
#суммируем тотал заказы по каждому городу 
df = pd.read_sql('''
select city, sum(count) as count_orders from (
select count(o.orderid) as count, c.city from client_info c
join order_info o on o.clientid = c.clientid 
join items_fashion i on i.item = o.item 
where i.cost > 5000
group by c.city
union all
select count(o.orderid) as count, c.city from client_info c
join order_info o on o.clientid = c.clientid 
join items_furniture it on it.item = o.item 
where it.cost > 20000
group by c.city
)
group by city
order by count_orders desc;

''', conn_sqlite)
df

,city,count_orders
0,Moscow,6
1,Sankt-Peterburg,4
2,Kazan,2


##  Финальный код

In [ ]:
#соединяем все таблицы выше в одну и высчитываем частоту
#группируем по самым частым заказам и получаем ответ - Санкт-Петербург
df = pd.read_sql('''
with tab as
(
select city, sum(count) as count_orders from 
    (select count(o.orderid) as count, c.city 
        from client_info c
    join order_info o
        on o.clientid = c.clientid 
    join items_fashion i 
        on i.item = o.item 
    where i.cost > 5000
    group by c.city
                  union all
    select count(o.orderid) as count, c.city
        from client_info c
    join order_info o 
        on o.clientid = c.clientid 
    join items_furniture it
        on it.item = o.item 
    where it.cost > 20000
    group by c.city)    
  group by city
  order by count_orders desc)
  select c.city, 
  count(distinct c.clientid) as count_clients, 
                     t.count_orders as orders, 
  (cast((t.count_orders) as float)) 
  / (cast(count(distinct c.clientid) as float)) 
                                     as Частота_заказа
  from client_info c
    left join tab t on t.city = c.city
    group by c.city
  order by Частота_заказа desc
  limit 1;

''', conn_sqlite)
df

,City,count_clients,orders,Частота_заказа
0,Sankt-Peterburg,9,4,0.444444
